# 2020 University of Minnesota Day of Data Python Notebook
# Exploration of American Community Survey data extracted from IPUMS-USA (https://ipums.org), a product of the U of M's Institute for Social Research and Data Innovation (ISRDI)

----------------------------------------------

## Python libraries, whether part of the standard set of Python libraries or from 3rd party sources, need to be imported. These are the libraries that we'll make use of in this notebook:
*  Pandas is a Python library for reading and manipulating tabular data. think "programmatic spreadsheets" 
*  Numpy is a number-processing library that pandas works closely with
*  BeautifulSoup is a library that can parse misc. markup languages, including XML
*  Altair is one of python's many data viz libariers

In [1]:
# pandas is a Python library for reading and manipulating tabular data. think "programatic spreadsheets"
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as BS
import altair as alt

## Let's start by reading in the data into a Pandas dataframe.
### The data file is in (gzipped) csv, which Pandas can read into a dataframe via its built-in read_csv() method

In [2]:
data = pd.read_csv("../data/usa_00071.csv.gz")

# the variable HHINCOME will show all 9s for no response, so let's change those to np.nan (which means "blank")
data["HHINCOME"] = data["HHINCOME"].replace(9999999, np.nan)

data

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,REGION,STATEFIP,METRO,CITY,...,HISPAN,HISPAND,EDUC,EDUCD,EMPSTAT,EMPSTATD,TRANWORK,EMPSTAT_SP,EMPSTATD_SP,TRANWORK_SP
0,2013,201301,21879,1813,143.0,2013000218791,42,2,2,210,...,0,0,10,101,3,30,0,1.0,10.0,70.0
1,2013,201301,21879,1813,143.0,2013000218791,42,2,2,210,...,0,0,10,101,1,10,70,3.0,30.0,0.0
2,2013,201301,21879,1813,143.0,2013000218791,42,2,2,210,...,0,0,5,50,3,30,0,NaN,NaN,NaN
3,2013,201301,21879,1813,143.0,2013000218791,42,2,2,210,...,0,0,2,26,0,0,0,NaN,NaN,NaN
4,2013,201301,21889,4944,446.0,2013000218891,42,2,2,210,...,0,0,10,101,1,10,10,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1712428,2019,201901,1425339,2019001408709,171.0,2019014253391,21,55,2,4130,...,0,0,1,12,0,0,0,NaN,NaN,NaN
1712429,2019,201901,1425339,2019001408709,171.0,2019014253391,21,55,2,4130,...,0,0,7,71,1,10,10,NaN,NaN,NaN
1712430,2019,201901,1425354,2019001409743,169.0,2019014253541,21,55,2,4130,...,0,0,6,64,3,30,0,NaN,NaN,NaN
1712431,2019,201901,1425363,2019001410132,172.0,2019014253631,21,55,2,4130,...,0,0,6,65,1,10,10,3.0,30.0,0.0


## In addition to the data, we have metadata that describes the data. This includes an XML file that maps the variables' numeric codes (how survey answers are represented in the data) to understandable labels.
### these two helper methods are for getting label information out of a provided XML file and into a codebook dict to translate data codes->labels

In [3]:
# this method takes in information on a given variable and returns a code-to-label dictionary for that variable
def parse_var_xml(var):
    var_values = {}
    for cat in var.find_all("catgry"):
        var_values[int(cat.catvalu.text)] = cat.labl.text
        
    return var_values

# Use Beautiful Soup to parse XML and send blocks of variable info to the parse_var_xml() method
# This method returns a codebook, which is a Pthon dict of dicts. Each top-level key is a variable, with values as a dice of code-to-label translations for that variable
def ipums_xml_to_var_dicts(xml_file):
    with open(xml_file, "r") as file:
        content = file.readlines()
        content = "".join(content)
        bs_content = BS(content, "lxml")
    variables = bs_content.find_all("var")
    codebook = {}
    for var in variables:
        codebook[var.get("name")] = parse_var_xml(var)
    
    return codebook

## Now that the methods are defined, to populate a codebook is simple: send the XML file to ipums_xml_to_var_dicts()

In [4]:
# create a dictionary of variable codes-to-labels for each variable
var_val_labels = ipums_xml_to_var_dicts("../syntax/usa_00071.xml")

## Let's have a look at one variable dictionary, TRANWORK (mode of transportation to get to work)

In [5]:
var_val_labels['TRANWORK']

{0: 'N/A',
 10: 'Auto, truck, or van',
 11: 'Auto',
 12: 'Driver',
 13: 'Passenger',
 14: 'Truck',
 15: 'Van',
 20: 'Motorcycle',
 31: 'Bus',
 32: 'Bus or trolley bus',
 33: 'Bus or streetcar',
 34: 'Light rail, streetcar, or trolley (Carro público in PR)',
 35: 'Streetcar or trolley car (publico in Puerto Rico, 2000)',
 36: 'Subway or elevated',
 37: 'Long-distance train or commuter train',
 38: 'Taxicab',
 39: 'Ferryboat',
 50: 'Bicycle',
 60: 'Walked only',
 70: 'Other',
 80: 'Worked at home'}

## using the var_val_labels dictionary, add columns for every variable's label value with the column name \<VARIABLE\>_lbl

In [6]:
for var in var_val_labels.keys():
    data[f"{var}_lbl"] = data[var].map(var_val_labels[var])
data

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,REGION,STATEFIP,METRO,CITY,...,HISPAN_lbl,HISPAND_lbl,EDUC_lbl,EDUCD_lbl,EMPSTAT_lbl,EMPSTATD_lbl,TRANWORK_lbl,EMPSTAT_SP_lbl,EMPSTATD_SP_lbl,TRANWORK_SP_lbl
0,2013,201301,21879,1813,143.0,2013000218791,42,2,2,210,...,Not Hispanic,Not Hispanic,4 years of college,Bachelor's degree,Not in labor force,Not in Labor Force,N/A,Employed,At work,Other
1,2013,201301,21879,1813,143.0,2013000218791,42,2,2,210,...,Not Hispanic,Not Hispanic,4 years of college,Bachelor's degree,Employed,At work,Other,Not in labor force,Not in Labor Force,N/A
2,2013,201301,21879,1813,143.0,2013000218791,42,2,2,210,...,Not Hispanic,Not Hispanic,Grade 11,Grade 11,Not in labor force,Not in Labor Force,N/A,NaN,NaN,NaN
3,2013,201301,21879,1813,143.0,2013000218791,42,2,2,210,...,Not Hispanic,Not Hispanic,"Grade 5, 6, 7, or 8",Grade 8,N/A,N/A,N/A,NaN,NaN,NaN
4,2013,201301,21889,4944,446.0,2013000218891,42,2,2,210,...,Not Hispanic,Not Hispanic,4 years of college,Bachelor's degree,Employed,At work,"Auto, truck, or van",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1712428,2019,201901,1425339,2019001408709,171.0,2019014253391,21,55,2,4130,...,Not Hispanic,Not Hispanic,Nursery school to grade 4,Kindergarten,N/A,N/A,N/A,NaN,NaN,NaN
1712429,2019,201901,1425339,2019001408709,171.0,2019014253391,21,55,2,4130,...,Not Hispanic,Not Hispanic,1 year of college,"1 or more years of college credit, no degree",Employed,At work,"Auto, truck, or van",NaN,NaN,NaN
1712430,2019,201901,1425354,2019001409743,169.0,2019014253541,21,55,2,4130,...,Not Hispanic,Not Hispanic,Grade 12,GED or alternative credential,Not in labor force,Not in Labor Force,N/A,NaN,NaN,NaN
1712431,2019,201901,1425363,2019001410132,172.0,2019014253631,21,55,2,4130,...,Not Hispanic,Not Hispanic,Grade 12,"Some college, but less than 1 year",Employed,At work,"Auto, truck, or van",Not in labor force,Not in Labor Force,N/A


## At this point we have a dataframe in which each row represents a single person

### We want to look at modes of transportation "prime age" workers use. So, first we subset our data down to those with an EMPSTAT of 1 (working) and those between 25 and 54 inclusive (prime working age)

In [7]:
workers = data[data['EMPSTAT']==1 & data["AGE"].between(25,54)]
workers

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,REGION,STATEFIP,METRO,CITY,...,HISPAN_lbl,HISPAND_lbl,EDUC_lbl,EDUCD_lbl,EMPSTAT_lbl,EMPSTATD_lbl,TRANWORK_lbl,EMPSTAT_SP_lbl,EMPSTATD_SP_lbl,TRANWORK_SP_lbl
1,2013,201301,21879,1813,143.0,2013000218791,42,2,2,210,...,Not Hispanic,Not Hispanic,4 years of college,Bachelor's degree,Employed,At work,Other,Not in labor force,Not in Labor Force,N/A
3,2013,201301,21879,1813,143.0,2013000218791,42,2,2,210,...,Not Hispanic,Not Hispanic,"Grade 5, 6, 7, or 8",Grade 8,N/A,N/A,N/A,NaN,NaN,NaN
4,2013,201301,21889,4944,446.0,2013000218891,42,2,2,210,...,Not Hispanic,Not Hispanic,4 years of college,Bachelor's degree,Employed,At work,"Auto, truck, or van",NaN,NaN,NaN
5,2013,201301,21890,4973,180.0,2013000218901,42,2,2,210,...,Not Hispanic,Not Hispanic,1 year of college,"1 or more years of college credit, no degree",Employed,At work,Other,Employed,At work,"Auto, truck, or van"
6,2013,201301,21890,4973,180.0,2013000218901,42,2,2,210,...,Not Hispanic,Not Hispanic,4 years of college,Bachelor's degree,Employed,At work,"Auto, truck, or van",Employed,At work,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1712425,2019,201901,1425339,2019001408709,171.0,2019014253391,21,55,2,4130,...,Not Hispanic,Not Hispanic,Grade 12,Regular high school diploma,Employed,At work,Worked at home,NaN,NaN,NaN
1712426,2019,201901,1425339,2019001408709,171.0,2019014253391,21,55,2,4130,...,Not Hispanic,Not Hispanic,Nursery school to grade 4,Kindergarten,N/A,N/A,N/A,NaN,NaN,NaN
1712427,2019,201901,1425339,2019001408709,171.0,2019014253391,21,55,2,4130,...,Not Hispanic,Not Hispanic,Nursery school to grade 4,Grade 4,N/A,N/A,N/A,NaN,NaN,NaN
1712428,2019,201901,1425339,2019001408709,171.0,2019014253391,21,55,2,4130,...,Not Hispanic,Not Hispanic,Nursery school to grade 4,Kindergarten,N/A,N/A,N/A,NaN,NaN,NaN


### To explore the data visually, create a dataframe that represents aggregate summary data
### Specifically, a dataframe in which each row represents a Year and City and the columns contain labels and counts for various variables
### To take the raw data and obtain counts of each City (CITY_lbl) by Year (YEAR) by type of transportation (TRANWORK_lbl), use the Pandas crosstab() method

In [8]:
df = pd.crosstab(index=[workers["YEAR"],
                        workers["CITY_lbl"],
                        workers["TRANWORK_lbl"]],
                 columns="count")
df

col_0                                                      count
YEAR CITY_lbl       TRANWORK_lbl                                
2013 Alexandria, VA Auto, truck, or van                      458
                    Bicycle                                   13
                    Bus or trolley bus                        40
                    Long-distance train or commuter train      2
                    Motorcycle                                 2
...                                                          ...
2019 Washington, DC Other                                     44
                    Subway or elevated                       656
                    Taxicab                                   28
                    Walked only                              370
                    Worked at home                           189

[2362 rows x 1 columns]

## This crosstab dataframe puts YEAR, CITY_lbl, and TRANWORK_lbl as indexes to the dataframe. We want them as columns, which is done with the dataframe's reset_index() method

In [9]:
df = df.reset_index()
df

col_0,YEAR,CITY_lbl,TRANWORK_lbl,count
0,2013,"Alexandria, VA","Auto, truck, or van",458
1,2013,"Alexandria, VA",Bicycle,13
2,2013,"Alexandria, VA",Bus or trolley bus,40
3,2013,"Alexandria, VA",Long-distance train or commuter train,2
4,2013,"Alexandria, VA",Motorcycle,2
...,...,...,...,...
2357,2019,"Washington, DC",Other,44
2358,2019,"Washington, DC",Subway or elevated,656
2359,2019,"Washington, DC",Taxicab,28
2360,2019,"Washington, DC",Walked only,370


## From here we need to go one step further. The data are in the format where each line shows a count for a particular type of transportation for a given year and city. We want to end up with the counts of the various transportation modes as columns, with one row per city/year
## This can be accomplished with a pivot table

In [10]:
table = df.pivot_table(index=["YEAR", "CITY_lbl"],
                       columns="TRANWORK_lbl",
                       values="count",
                       aggfunc='sum',
                       margins=True,
                       fill_value=0)
table

TRANWORK_lbl          Auto, truck, or van  Bicycle   Bus  Bus or trolley bus  \
YEAR CITY_lbl                                                                  
2013 Alexandria, VA                   458       13     0                  40   
     Anchorage, AK                    341        4     0                  13   
     Billings, MT                     184        4     0                   5   
     Boston, MA                      1008       41     0                 248   
     Chicago, IL                     4380      119     0                 860   
...                                   ...      ...   ...                 ...   
2019 Seattle, WA                     1476      105   573                   0   
     Sioux Falls, SD                  364        0     2                   0   
     Toledo, OH                       494        1    12                   0   
     Washington, DC                  1008      162   270                   0   
All                                321739    12011  6869               43222   

TRANWORK_lbl          Ferryboat  \
YEAR CITY_lbl                     
2013 Alexandria, VA           0   
     Anchorage, AK            0   
     Billings, MT             0   
     Boston, MA               0   
     Chicago, IL              1   
...                         ...   
2019 Seattle, WA              6   
     Sioux Falls, SD          0   
     Toledo, OH               0   
     Washington, DC           0   
All                         718   

TRANWORK_lbl          Light rail, streetcar, or trolley (Carro público in PR)  \
YEAR CITY_lbl                                                                   
2013 Alexandria, VA                                                   0         
     Anchorage, AK                                                    0         
     Billings, MT                                                     0         
     Boston, MA                                                       0         
     Chicago, IL                                                      0         
...                                                                 ...         
2019 Seattle, WA                                                     83         
     Sioux Falls, SD                                                  0         
     Toledo, OH                                                       0         
     Washington, DC                                                   6         
All                                                                 536         

TRANWORK_lbl          Long-distance train or commuter train  Motorcycle  \
YEAR CITY_lbl                                                             
2013 Alexandria, VA                                       2           2   
     Anchorage, AK                                        0           2   
     Billings, MT                                         0           1   
     Boston, MA                                          34           4   
     Chicago, IL                                        163           7   
...                                                     ...         ...   
2019 Seattle, WA                                          3          13   
     Sioux Falls, SD                                      0           0   
     Toledo, OH                                           0           2   
     Washington, DC                                      14           8   
All                                                    5640        1424   

TRANWORK_lbl             N/A  Other  \
YEAR CITY_lbl                         
2013 Alexandria, VA      243      7   
     Anchorage, AK       286     10   
     Billings, MT        143      0   
     Boston, MA          911     13   
     Chicago, IL        4330     60   
...                      ...    ...   
2019 Seattle, WA         954     15   
     Sioux Falls, SD     219      0   
     Toledo, OH          390      2   
     Washington, DC      881     44   
All  

## A couple pieces of cleanup. 
* First we want the indexes as columns, so as before we reset_index()
* Second, we do not need the "All" row that represents the counts of all rows

In [12]:
table = table.reset_index()
# Drop the All row (YEAR=="All")
table = table[table["YEAR"]!="All"]
table

TRANWORK_lbl,YEAR,CITY_lbl,"Auto, truck, or van",Bicycle,Bus,Bus or trolley bus,Ferryboat,"Light rail, streetcar, or trolley (Carro público in PR)",Long-distance train or commuter train,Motorcycle,...,% N/A,% Other,% Subway or elevated,% Taxicab,% Walked only,% Worked at home,"% Streetcar or trolley car (publico in Puerto Rico, 2000)",% Ferryboat,% Bus,"% Light rail, streetcar, or trolley (Carro público in PR)"
0,2013,"Alexandria, VA",458,13,0,40,0,0,2,2,...,0.269401,0.007761,0.099778,0.001109,0.016630,0.034368,0.000000,0.000000,0.000000,0.000000
1,2013,"Anchorage, AK",341,4,0,13,0,0,0,2,...,0.415094,0.014514,0.000000,0.000000,0.027576,0.020319,0.000000,0.000000,0.000000,0.000000
2,2013,"Billings, MT",184,4,0,5,0,0,0,1,...,0.405099,0.000000,0.000000,0.000000,0.005666,0.039660,0.000000,0.000000,0.000000,0.000000
3,2013,"Boston, MA",1008,41,0,248,0,0,34,4,...,0.305807,0.004364,0.124203,0.003357,0.085935,0.024169,0.004028,0.000000,0.000000,0.000000
4,2013,"Chicago, IL",4380,119,0,860,1,0,163,7,...,0.371100,0.005142,0.085019,0.002143,0.033339,0.028282,0.001028,0.000086,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,2019,"San Francisco, CA",1258,171,736,0,0,78,77,39,...,0.229424,0.013456,0.069522,0.012559,0.090828,0.055169,0.000000,0.000000,0.165059,0.017493
220,2019,"Seattle, WA",1476,105,573,0,6,83,3,13,...,0.258887,0.004071,0.004342,0.002442,0.061058,0.056174,0.000000,0.001628,0.155495,0.022524
221,2019,"Sioux Falls, SD",364,0,2,0,0,0,0,0,...,0.365000,0.000000,0.000000,0.000000,0.008333,0.016667,0.000000,0.000000,0.003333,0.000000
222,2019,"Toledo, OH",494,1,12,0,0,0,0,2,...,0.418904,0.002148,0.000000,0.001074,0.018260,0.012889,0.000000,0.000000,0.012889,0.000000


## To compare across cities, raw counts are not sufficient (as each city have different amount of survey respondents)
### Create new columns that represent the RATIO of a given transporation mode to the total count of responses

In [13]:
for col in df["TRANWORK_lbl"].unique():
    table[f"% {col}"] = (table[col] / table["All"])
table

TRANWORK_lbl,YEAR,CITY_lbl,"Auto, truck, or van",Bicycle,Bus,Bus or trolley bus,Ferryboat,"Light rail, streetcar, or trolley (Carro público in PR)",Long-distance train or commuter train,Motorcycle,...,% N/A,% Other,% Subway or elevated,% Taxicab,% Walked only,% Worked at home,"% Streetcar or trolley car (publico in Puerto Rico, 2000)",% Ferryboat,% Bus,"% Light rail, streetcar, or trolley (Carro público in PR)"
0,2013,"Alexandria, VA",458,13,0,40,0,0,2,2,...,0.269401,0.007761,0.099778,0.001109,0.016630,0.034368,0.000000,0.000000,0.000000,0.000000
1,2013,"Anchorage, AK",341,4,0,13,0,0,0,2,...,0.415094,0.014514,0.000000,0.000000,0.027576,0.020319,0.000000,0.000000,0.000000,0.000000
2,2013,"Billings, MT",184,4,0,5,0,0,0,1,...,0.405099,0.000000,0.000000,0.000000,0.005666,0.039660,0.000000,0.000000,0.000000,0.000000
3,2013,"Boston, MA",1008,41,0,248,0,0,34,4,...,0.305807,0.004364,0.124203,0.003357,0.085935,0.024169,0.004028,0.000000,0.000000,0.000000
4,2013,"Chicago, IL",4380,119,0,860,1,0,163,7,...,0.371100,0.005142,0.085019,0.002143,0.033339,0.028282,0.001028,0.000086,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,2019,"San Francisco, CA",1258,171,736,0,0,78,77,39,...,0.229424,0.013456,0.069522,0.012559,0.090828,0.055169,0.000000,0.000000,0.165059,0.017493
220,2019,"Seattle, WA",1476,105,573,0,6,83,3,13,...,0.258887,0.004071,0.004342,0.002442,0.061058,0.056174,0.000000,0.001628,0.155495,0.022524
221,2019,"Sioux Falls, SD",364,0,2,0,0,0,0,0,...,0.365000,0.000000,0.000000,0.000000,0.008333,0.016667,0.000000,0.000000,0.003333,0.000000
222,2019,"Toledo, OH",494,1,12,0,0,0,0,2,...,0.418904,0.002148,0.000000,0.001074,0.018260,0.012889,0.000000,0.000000,0.012889,0.000000


# Now Let's graph some data!

## First, we can try to compare percent of working population working from home

In [14]:
line = alt.Chart(table).mark_line(interpolate="natural").encode(
    alt.X('YEAR:O'),
    alt.Y('% Worked at home:Q'),
    alt.Color('CITY_lbl'),
).properties(width=500, height=500)
line

alt.Chart(...)

## Well. That's a mess.
## How about we scale this back a bit...Just % Worked at home for the year 2019, and display it as a vertical bar chart

In [19]:
# Start with just a 1 year
one_year = table[table["YEAR"]==2019]
bar = alt.Chart(one_year, title="2019").mark_bar().encode(
        alt.X('% Worked at home'),
        alt.Y('CITY_lbl:N'),
    )
bar

alt.Chart(...)

## Better! But, hard to compare without these ranked by value...
## Use sort="-x" on the Y axis to sort in descending order

In [20]:
# Sort based on X value
one_year = table[table["YEAR"]==2019]
bar = alt.Chart(one_year, title="2019").mark_bar().encode(
        alt.X('% Worked at home'),
        alt.Y('CITY_lbl:N', sort="-x"),
    )
bar

alt.Chart(...)

## Cool. Cool Cool Cool. Now that we've got 2019 displaying nicely, let's show every year with the .hconcat() multiple chart feature

In [21]:
# Multiple years
charts = alt.hconcat()
for y in table["YEAR"].unique():
    one_year = table[table["YEAR"]==y]
    bar = alt.Chart(one_year, title=str(y)).mark_bar().encode(
        alt.X('% Worked at home'),
        alt.Y('CITY_lbl:N', sort="-x"),
    ).properties(width=150)
    charts |= bar
    
charts

alt.HConcatChart(...)

## Lovely! Now that we're to this point, let's do some tidying up.
* Add a title for the set of charts
* The Y-axis label CITY_Lbl is unneeded
* Whoops, % Worked at home is displaying ratios not percentages. Fix that formatting
* Finally, make the X-axis range across each chart consistent by finding the max x value across all years and create the scale based on that

In [25]:
# Consistent X axis range
charts = alt.hconcat(title="City rankings for % Working at Home")
max_pct = table["% Worked at home"].max()
for y in table["YEAR"].unique():
    bar = alt.Chart(table[table["YEAR"]==y], title=str(y)).mark_bar().encode(
        alt.X(
            '% Worked at home',
            axis=alt.Axis(format="%"),
            scale=alt.Scale(domain=(0, max_pct)),
        ),
        alt.Y('CITY_lbl:N', sort="-x", title=None),
    ).properties(width=150)
    charts |= bar
    
charts

alt.HConcatChart(...)

## Excellent!
## Now that we're nicely cleaned up, let's highlight a city of interest using alt.condition()
## Let's take a look at the great city of Minneapolis by making its bar yellow across each chart

In [28]:
# conditional coloring of one bar
charts = alt.hconcat(title="City rankings for % Working at Home (age 25-55)")
max_pct = table["% Worked at home"].max()
for y in table["YEAR"].unique():
    bar = alt.Chart(table[table["YEAR"]==y], title=str(y)).mark_bar().encode(
        alt.X(
            '% Worked at home',
            axis=alt.Axis(format="%"),
            scale=alt.Scale(domain=(0, max_pct)),
        ),
        alt.Y('CITY_lbl:N', sort="-x", title=None),
        color=alt.condition(
            alt.datum.CITY_lbl == "Minneapolis, MN",
            alt.value('orange'),
            alt.value('steelblue'),
        )
    ).properties(width=150)
    charts |= bar
    
charts

alt.HConcatChart(...)

## Using the same code but pointing to % Bicycle, we can produce the same type of graphs for different variables

In [29]:
# conditional coloring of one bar
charts = alt.hconcat(title="City rankings for % Biking to work (age 25-55)")
max_pct = table["% Bicycle"].max()
for y in table["YEAR"].unique():
    bar = alt.Chart(table[table["YEAR"]==y], title=str(y)).mark_bar().encode(
        alt.X(
            '% Bicycle',
            axis=alt.Axis(format="%"),
            scale=alt.Scale(domain=(0, max_pct)),
        ),
        alt.Y('CITY_lbl:N', sort="-x", title=None),
        color=alt.condition(
            alt.datum.CITY_lbl == "Minneapolis, MN",
            alt.value('orange'),
            alt.value('steelblue'),
        )
    ).properties(width=150)
    charts |= bar
    
charts

alt.HConcatChart(...)

---------------------------------------------------------
---------------------------------------------------------
---------------------------------------------------------